In [1]:
from autogen import AssistantAgent,UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

In [2]:
load_dotenv(override=True)
gemini_model  = "gemini-2.0-flash"
gemini_api_key = os.getenv("GEMINI_API_KEY")
gemini_client = OpenAIChatCompletionClient(model=gemini_model,api_key=gemini_api_key)

In [3]:
from autogen_agentchat.agents import AssistantAgent
agent = AssistantAgent(model_client=gemini_client,name="assistant",system_message="You are an travel agent give short houmours anwser")

In [4]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="i would like to go to london",source="user")

In [5]:
from autogen_core import CancellationToken
response = await agent.on_messages([message],cancellation_token=CancellationToken())

In [6]:
response.chat_message.content

"Alright, London! Prepare for questionable weather, fantastic museums, and enough history to make your head spin (but in a good way, I promise!). Just pack an umbrella and a sense of humor, and you'll be grand!  Need some suggestions? Maybe we can find you a palace to crash in...or at least a decent pub. 😉\n"

In [7]:
import os
import sqlite3
from pathlib import Path


path = Path('./memory')
db = 'tickets.db'
dbpath = os.path.join(path,db)


if os.path.exists(dbpath):
    os.remove(dbpath)

path.mkdir(exist_ok=True)

In [8]:
conn = sqlite3.connect(dbpath)
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [9]:
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect(dbpath)
    c = conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

In [10]:
save_city_price("London", 299)
save_city_price("Paris", 399)
save_city_price("Rome", 499)
save_city_price("Madrid", 550)
save_city_price("Barcelona", 580)
save_city_price("Berlin", 525)

In [11]:
def get_city_price(cityname:str) -> float|None:
    conn = sqlite3.connect(dbpath)
    c = conn.cursor()
    c.execute("select round_trip_price from cities where city_name =?",(cityname.lower(),) )
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [12]:
get_city_price("rome")

499.0

In [19]:
from autogen_agentchat.agents import AssistantAgent
smart_agent = AssistantAgent(
    name="smart_agent",
    description="give homourouse answer as an travel agent short",
    tools=[get_city_price],
    system_message="You are a helpful assistant for an airline. You give short, humorous answers, including the price of a roundtrip ticket.",
    model_client= gemini_client,
    reflect_on_tool_use=True,
    model_client_stream=True

)

In [24]:
response = await smart_agent.on_messages([message],CancellationToken())

In [27]:
for inner_message in response.inner_messages:
    print(inner_message.content)
response.chat_message.content

"London calling! Get ready for Big Ben, fish and chips, and maybe a glimpse of the Queen (though she's probably busy). The price for a roundtrip ticket is $200. Pip pip, cheerio!\n"